In [1]:
import os

import tensorflow as tf

/home/sinho/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [3]:
df_train = pd.read_csv('data/fashion_train.csv')

In [4]:

df_train_X = df_train.iloc[:, 1:]
df_train_y = df_train.iloc[:,[0]]

In [5]:
df_train_X.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,3,...,23,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,141,73,0,0,107,63,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,19,35,7,14,0,0,0


In [6]:
train_X = df_train_X

In [7]:
train_y = pd.get_dummies(df_train_y,columns=['label'],sparse=True)

In [8]:
df_test = pd.read_csv('data/fashion_test.csv')

In [9]:
df_test.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,1,0,0,0,0,0,0,0,0,0,...,208,174,128,0,0,0,0,0,0,0
1,6,0,0,0,0,0,0,1,0,1,...,49,40,3,0,40,66,11,0,0,0
2,8,0,0,0,0,0,0,0,0,0,...,128,126,126,120,129,75,0,0,0,0
3,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,123,145,152,206,58,0,0,0,0,0


In [10]:
df_test_X = df_test.iloc[:,1:]
df_test_y = df_test.iloc[:,[0]]

In [11]:
test_X = df_test_X
test_y = pd.get_dummies(df_test_y, columns=['label'], sparse = True)

In [12]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784] , name = 'img')
y  = tf.placeholder(tf.float32, [None, 10], name = 'label')

W1 = tf.Variable(tf.random_normal([784, 256], stddev = 0.01))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_normal([256 , 256], stddev = 0.01))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_normal([256, 10], stddev = 0.01))
model = tf.nn.softmax(tf.matmul(L2, W3))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = model, labels = y ))
train_op = tf.train.AdamOptimizer(0.001).minimize(cost)

In [13]:
print(W3)
print(model)

<tf.Variable 'Variable_2:0' shape=(256, 10) dtype=float32_ref>
Tensor("Softmax:0", shape=(?, 10), dtype=float32)


In [14]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(1000):
        sess.run(train_op, feed_dict = {X : train_X , y : train_y})
        if (epoch + 1) % 100 == 0:
            print('epoch : {} , {}'.format(epoch +1 , sess.run(cost, feed_dict = {X : train_X , y : train_y})))
    pred = tf.argmax(model, axis = 1)
    target = tf.argmax(train_y, axis = 1)
    print('실제값: ', sess.run(target, feed_dict = {y: train_y}))
    print('예측값: ', sess.run(pred, feed_dict = {X: train_X}))
    
    correct = tf.equal(pred, target)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    print('정확도: {}'.format(sess.run(accuracy * 100, feed_dict = {X : train_X, y : train_y})))
    
    print('--' * 30)
    print('Test')
    print()
    
    is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    print('정확도: {}'.format(sess.run(acc * 100, feed_dict = {X : test_X, y : test_y})))

epoch : 100 , 1.6470341682434082
epoch : 200 , 1.6090092658996582
epoch : 300 , 1.569936752319336
epoch : 400 , 1.5369592905044556
epoch : 500 , 1.5160598754882812
epoch : 600 , 1.528292179107666
epoch : 700 , 1.504975438117981
epoch : 800 , 1.5046427249908447
epoch : 900 , 1.5025359392166138
epoch : 1000 , 1.5019475221633911
실제값:  [5 0 6 ..., 0 9 1]
예측값:  [5 0 6 ..., 0 9 1]
정확도: 95.91190338134766
------------------------------------------------------------
Test

정확도: 89.0
